In [1]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# webdriver_options = webdriver.ChromeOptions()
# webdriver_options.add_argument('headless') 
driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver.exe')

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


## 멜론 연도별 top 100 가사 크롤링
* 연령제한 걸려있는 노래가 있어 모두 가져오기 위해서는 로그인 하고 돌려야 함 
* 로그인 하지 않아도 에러는 나지 않음! 

In [17]:
all_data=pd.DataFrame()
start = 2010 #시작 연도
end = 2014 #끝나는 연도
for year in range(start,end+1) :  #가져올 연도 지정 
    
    url=f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={year}'
    driver.get(url)
    time.sleep(2)
    html=driver.page_source
    soup = BeautifulSoup(html, "lxml")


    ###### 상세페이지 정보 가져오기 
    detail_num=[]
    #1~50위 노래 상세페이지 num 저장
    for rank in range(50) :
        detail=soup.select('tr.lst50>td.t_left>div.wrap>a')[rank]['onclick']
        num=re.sub('[^0-9]','',detail)
        detail_num.append(num) #숫자만 남기기

    #50~100위 노래 상세페이지 num 저장
    for rank in range(50) :
        detail=soup.select('tr.lst100>td.t_left>div.wrap>a')[rank]['onclick']
        num=re.sub('[^0-9]','',detail)
        detail_num.append(num) #숫자만 남기기    

    time.sleep(1)

    ###### 20xx연도 top  n 가사 크롤링   
    dataset=pd.DataFrame()
    top = 100 # 상위 몇곡 가져올지 지정 가능
    for i in range(top): 
        try :
            driver.get('https://www.melon.com/song/detail.htm?songId={detail_num}'.format(detail_num=detail_num[i]))
            time.sleep(2)
            #가사,노래제목,가수 크롤링 함수 

            lyric = driver.find_elements_by_class_name('lyric')[0].text #가사
            time.sleep(0.5)
            song_name = driver.find_elements_by_class_name('song_name')[0].text #노래제목 
            time.sleep(0.5)
            artist = driver.find_elements_by_class_name('artist')[0].text #가수 이름

            lyrics = list(filter(None,lyric.split('\n'))) # split 후 공백제거 
            df=pd.DataFrame({'lyrics':lyrics})
            df['artist']=artist
            df['song_name']=song_name
            df['Year']=year #연도추가
            dataset=dataset.append(df) 
        except :
            pass
        
        time.sleep(0.5)
    print(year,'년도 노래들을 모두 가져왔습니다.')    
    all_data=all_data.append(dataset)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


2010 년도 노래들을 모두 가져왔습니다.
2011 년도 노래들을 모두 가져왔습니다.
2012 년도 노래들을 모두 가져왔습니다.
2013 년도 노래들을 모두 가져왔습니다.
2014 년도 노래들을 모두 가져왔습니다.


In [18]:
data2=all_data

In [26]:
data2

,lyrics,artist,song_name,Year
0,You don't know me,미쓰에이,Bad Girl Good Girl,2010
1,You don't know me,미쓰에이,Bad Girl Good Girl,2010
2,You don't know me,미쓰에이,Bad Girl Good Girl,2010
3,You don't know me,미쓰에이,Bad Girl Good Girl,2010
4,So shut off boy,미쓰에이,Bad Girl Good Girl,2010
...,...,...,...,...
68,떡볶이 떡볶이 순대 순대,박명수,"명수네 떡볶이 (Feat. 김예림, UL)",2014
69,말해봐 다 있어,박명수,"명수네 떡볶이 (Feat. 김예림, UL)",2014
70,떡볶이 떡볶이 튀김 튀김,박명수,"명수네 떡볶이 (Feat. 김예림, UL)",2014
71,떡볶이 떡볶이 김밥,박명수,"명수네 떡볶이 (Feat. 김예림, UL)",2014


In [15]:
#data1.to_csv('멜론가사크롤링1.csv',index=False)
#data2.to_csv('멜론가사크롤링2.csv',index=False)

* 혹시 모를 에러 방지를 위해 2번 나눠서 크롤링 진행

In [21]:
print(' 2010년부터 2014년까지 가져온 노래 개수는',len(data2.song_name.unique()),'곡 입니다.') #노래개수
print(' 2015년부터 2021년까지 가져온 노래 개수는',len(data1.song_name.unique()),'곡 입니다.') #노래개수

 2010년부터 2014년까지 가져온 노래 개수는 475 곡 입니다.
 2015년부터 2021년까지 가져온 노래 개수는 567 곡 입니다.


In [43]:
data=pd.concat([data2,data1])
print(' 2010년부터 2021년까지 가져온 노래 개수는',len(data.song_name.unique()),'곡 입니다.')

 2010년부터 2021년까지 가져온 노래 개수는 1025 곡 입니다.


In [ ]:
data.to_csv('멜론가사크롤링2010to2021.csv',index=False)

In [33]:
#data=data.set_index('song_name')  # index 노래 제목으로 설정

In [54]:
data

,lyrics,artist,song_name,Year
0,You don't know me,미쓰에이,Bad Girl Good Girl,2010
1,You don't know me,미쓰에이,Bad Girl Good Girl,2010
2,You don't know me,미쓰에이,Bad Girl Good Girl,2010
3,You don't know me,미쓰에이,Bad Girl Good Girl,2010
4,So shut off boy,미쓰에이,Bad Girl Good Girl,2010
...,...,...,...,...
90,끝이 아니길,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021
91,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021
92,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021
93,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021
